In [1]:
from flask import Flask, jsonify, send_from_directory
import cv2
import mediapipe as mp
import numpy as np
import math
import time
import pyautogui
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=2, min_detection_confidence=0.7, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Function to calculate angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - b[0])
    angle = np.abs(radians * 180.0 / np.pi)

    if angle > 180.0:
        angle = 360 - angle

    return angle

def run_gesture_model():
    cap = cv2.VideoCapture(0)
    last_action_time = time.time()
    action_delay = 2

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        result = hands.process(rgb_frame)

        if result.multi_hand_landmarks and result.multi_handedness:
            hand_landmarks_list = result.multi_hand_landmarks
            handedness_list = result.multi_handedness
            
            thumbs_detected = 0
            index_finger_detected = {"Left": False, "Right": False}
            thumb_detected = {"Left": False, "Right": False}
            
            for hand_landmarks, handedness in zip(hand_landmarks_list, handedness_list):
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                landmarks = [[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark]

                index_angle = calculate_angle(landmarks[8], landmarks[7], landmarks[6])
                thumb_angle = calculate_angle(landmarks[4], landmarks[3], landmarks[2])
                
                hand_label = handedness.classification[0].label
                if index_angle > 160:
                    index_finger_detected[hand_label] = True
                
                if thumb_angle > 150:
                    thumbs_detected += 1
                    thumb_detected[hand_label] = True

                cv2.putText(frame, f'{hand_label} Index Angle: {int(index_angle)}', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)
                cv2.putText(frame, f'{hand_label} Thumb Angle: {int(thumb_angle)}', (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

            current_time = time.time()
            if thumbs_detected == 2 and current_time - last_action_time > action_delay:
                pyautogui.hotkey('alt', 'f4')
                last_action_time = current_time
            elif thumb_detected["Left"] or thumb_detected["Right"] and current_time - last_action_time > action_delay:
                pyautogui.press('space')
                last_action_time = current_time
            elif index_finger_detected["Right"] and current_time - last_action_time > action_delay:
                pyautogui.hotkey('ctrl', 'right')
                last_action_time = current_time
            elif index_finger_detected["Left"] and current_time - last_action_time > action_delay:
                pyautogui.hotkey('ctrl', 'left')
                last_action_time = current_time

        cv2.imshow('Hand Tracking', frame)
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()
run_gesture_model()
